## Upload clean dataset 'creditriskdf1' with outliers removed.

In [80]:
val creditRiskdf1 = spark.read.option("header","true").csv("creditRiskdf1_1.csv")
val creditRiskdf2 = spark.read.option("header","true").csv("creditRiskdf1_2.csv")
val creditRiskdf3 = spark.read.option("header","true").csv("creditRiskdf1_3.csv")

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


In [81]:
import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.Pipeline

import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.Pipeline


In [82]:
val creditRiskdf1and2 = creditRiskdf1.union(creditRiskdf2)
var creditRiskdf = creditRiskdf1and2.union(creditRiskdf3)

creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


## Split the dataframe to extract a test dataframe

In [84]:
val creditRiskFeatures = creditRiskdf.columns
val features = creditRiskFeatures.filter(!_.contains("TARGET")).filter(!_.contains("SK_ID_CURR")) //Make sure we exclude the ID column and label column

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_ID_PUBLISH, previous_loans_CNT_PAYMENT_mean, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, DAYS_REGISTRATION, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_mean, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, previous_loans_DAYS_FIRST_DUE_min, client_cash_SK_DPD_DEF_sum_max, bureau_AMT_CREDIT_SUM_LIMIT_mean, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, previous_loans_PRODUCT_COMBINATION_Cash X-Sell: low_count_norm, ...

In [112]:
val creditRiskFeatures = creditRiskdf.columns
for (colName<-creditRiskFeatures){
     |   creditRiskdf=creditRiskdf.withColumn(colName,col(colName).cast("Float"))
     | }

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_ID_PUBLISH, previous_loans_CNT_PAYMENT_mean, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, DAYS_REGISTRATION, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_mean, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, previous_loans_DAYS_FIRST_DUE_min, client_cash_SK_DPD_DEF_sum_max, bureau_AMT_CREDIT_SUM_LIMIT_mean, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, previous_loans_PRODUCT_COMBINATION_Cash X-Sell: low_count_norm, ...

In [113]:
var Array(train_df,test_df,validation_df)=creditRiskdf.randomSplit(
    Array(0.7,0.2,0.1))

train_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
test_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
validation_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


## Create dataframe 'df13' with same amount of rows for TARGET==0 and TARGET==1

In [86]:
creditRiskdf.count()

res38: Long = 183875


In [87]:
val df9=creditRiskdf.filter($"TARGET" === 1.0)
df9.count()

df9: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
res39: Long = 13312


In [88]:
val df10=creditRiskdf.filter($"TARGET" === 0.0)
df10.count()

df10: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
res40: Long = 170563


In [99]:
//Now from 33224 rows of TARGET==0, we randomly extract 2300 rows and combine it with df9
//To get a dataframe of 2300 target==1 and 2300 target ==0
var Array(df11,df12)=df10.randomSplit(
    Array(0.078,0.922))

df11: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
df12: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [100]:
val df13=df9.union(df11)

df13: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [101]:
df13.count()

res47: Long = 26446


In [102]:
df13.filter($"TARGET" === 0.0).count()


res48: Long = 13134


In [103]:
df13.filter($"TARGET" === 1.0).count()

res49: Long = 13312


## Train the model with df13 instead of train_df

In [104]:
val assembler = new VectorAssembler().setInputCols(features).setOutputCol("features")
val labelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol("label")
val model = new LogisticRegression()

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_015fcd2452b8
labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_acca6523988e
model: org.apache.spark.ml.classification.LogisticRegression = logreg_f38d23fd9ba4


In [105]:
val assembler = new VectorAssembler().setInputCols(features).setOutputCol("features")
val df2 = assembler.transform(df13)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_5ebc6a77e6fc
df2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 187 more fields]


In [106]:
val labelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol("label")
val df3 = labelIndexer.fit(df2).transform(df2)

labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_2e1d53c13f82
df3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 188 more fields]


In [107]:
val model = new LogisticRegression().fit(df3)
val predictions2 = model.transform(df3)

model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_1757437c3634, numClasses = 2, numFeatures = 186
predictions2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]


## Test the model with test_df which have much more target==0 than target==1

In [108]:
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics


In [109]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


In [110]:
val assembler = new VectorAssembler().setInputCols(features).setOutputCol("features")
val labelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol("label")
val model = new LogisticRegression()
val lrPipeline = new Pipeline().setStages(Array(labelIndexer, assembler, model))

val lrModel = lrPipeline.fit(df13)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_657f6e899512
labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_5498f6595964
model: org.apache.spark.ml.classification.LogisticRegression = logreg_87a1a325970d
lrPipeline: org.apache.spark.ml.Pipeline = pipeline_f8a05caf6f32
lrModel: org.apache.spark.ml.PipelineModel = pipeline_f8a05caf6f32


In [114]:
val predictions3 = lrModel.transform(test_df)

predictions3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]


In [115]:
val predictionLabelsRDD2 = predictions3.select("prediction", "label").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics2 = new BinaryClassificationMetrics(predictionLabelsRDD2)
println("area under the receiver operating characteristic (ROC) curve : " + binMetrics1.areaUnderROC)

area under the receiver operating characteristic (ROC) curve : 0.7051602943316483


predictionLabelsRDD2: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[7917] at map at <console>:81
binMetrics2: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@6911c705


In [116]:
val lp = predictions3.select( "label", "prediction")
val counttotal = predictions3.count()
val correct = lp.filter($"label" === $"prediction").count() //correct prediction
val wrong = lp.filter(not($"label" === $"prediction")).count() //wrong prediction
val truedefault=lp.filter($"label"===1.0).count// Number of target==1
val truep = lp.filter($"prediction" === 0.0).filter($"label" === $"prediction").count() //Correct prediction of label=0
val truen = lp.filter($"prediction" === 1.0).filter($"label" === $"prediction").count() //Correct prediction of label=1
val falseN = lp.filter($"prediction" === 0.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when label=1
val falseP = lp.filter($"prediction" === 1.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when lable =0
val ratioWrong=wrong.toDouble/counttotal.toDouble //Wrong prediction ratio
val ratioCorrect=correct.toDouble/counttotal.toDouble //Correct predicition ratio
val ratiotrueN=truen.toDouble/truedefault.toDouble //Correct prediction ratio for target ==1

lp: org.apache.spark.sql.DataFrame = [label: double, prediction: double]
counttotal: Long = 36778
correct: Long = 25653
wrong: Long = 11125
truedefault: Long = 34054
truep: Long = 1882
truen: Long = 23771
falseN: Long = 10283
falseP: Long = 842
ratioWrong: Double = 0.3024906193920278
ratioCorrect: Double = 0.6975093806079722
ratiotrueN: Double = 0.6980384095847771


We can see that the model can predict 69% of the default. and 69% of the non-default

In [78]:
//test of total accuracy
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions3)
println("Test set accuracy = " + accuracy)

Test set accuracy = 0.6932293117860128


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_35eb89cda82e
accuracy: Double = 0.6932293117860128


## Test the model with df13

In [117]:
val predictionLabelsRDD1 = predictions2.select("prediction", "label").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics1 = new BinaryClassificationMetrics(predictionLabelsRDD1)
println("area under the receiver operating characteristic (ROC) curve : " + binMetrics1.areaUnderROC)

area under the receiver operating characteristic (ROC) curve : 0.6936940928235291


predictionLabelsRDD1: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[8111] at map at <console>:80
binMetrics1: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@559ae297


In [118]:
val lp = predictions2.select( "label", "prediction")
val counttotal = predictions2.count()
val correct = lp.filter($"label" === $"prediction").count() //correct prediction
val wrong = lp.filter(not($"label" === $"prediction")).count() //wrong prediction
val truep = lp.filter($"prediction" === 0.0).filter($"label" === $"prediction").count() //Correct prediction of label=0
val truen = lp.filter($"prediction" === 1.0).filter($"label" === $"prediction").count() //Correct prediction of label=1
val falseN = lp.filter($"prediction" === 0.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when label=1
val falseP = lp.filter($"prediction" === 1.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when lable =0
val ratioWrong=wrong.toDouble/counttotal.toDouble //Wrong prediction ratio
val ratioCorrect=correct.toDouble/counttotal.toDouble //Correct predicition ratio

lp: org.apache.spark.sql.DataFrame = [label: double, prediction: double]
counttotal: Long = 26446
correct: Long = 18345
wrong: Long = 8101
truep: Long = 9202
truen: Long = 9143
falseN: Long = 3991
falseP: Long = 4110
ratioWrong: Double = 0.30632231717462
ratioCorrect: Double = 0.69367768282538


In [35]:
predictions2.select ("SK_ID_CURR","features", "label","probability", "prediction").show()

+----------+--------------------+-----+--------------------+----------+
|SK_ID_CURR|            features|label|         probability|prediction|
+----------+--------------------+-----+--------------------+----------+
|  100049.0|(186,[0,1,2,3,4,5...|  0.0|[0.41297183801247...|       1.0|
|  100902.0|(186,[0,1,2,3,4,5...|  0.0|[0.40944020354736...|       1.0|
|  101042.0|(186,[0,1,2,3,4,5...|  0.0|[0.38802373681337...|       1.0|
|  101197.0|(186,[0,1,2,3,4,5...|  0.0|[0.63103137138405...|       0.0|
|  101277.0|(186,[0,1,2,3,4,5...|  0.0|[0.91460324467911...|       0.0|
|  101286.0|(186,[0,1,2,3,4,5...|  0.0|[0.60208368726355...|       0.0|
|  101495.0|(186,[0,1,2,3,4,5...|  0.0|[0.51719934824546...|       0.0|
|  101859.0|(186,[0,1,2,3,4,5...|  0.0|[0.63584047678167...|       0.0|
|  101980.0|(186,[0,1,2,3,4,5...|  0.0|[0.72854178873096...|       0.0|
|  102003.0|(186,[0,1,2,3,4,5...|  0.0|[0.78686212416355...|       0.0|
|  102025.0|(186,[0,1,2,3,4,5...|  0.0|[0.67750359233151...|    

In [120]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


In [121]:
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions2)
println("Test set accuracy = " + accuracy)

Test set accuracy = 0.69367768282538


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_88fbc19109f8
accuracy: Double = 0.69367768282538
